In [1]:
import os, sys
sys.path.insert(0, os.environ['PROJECT_PATH'])

In [2]:
import pandas as pd, requests, datetime

In [3]:
from config.resources import path_to

In [15]:
funding_rounds_df = pd.read_csv(path_to['with_msa_csv'].format(node='funding_rounds'), encoding='utf-8')
funding_rounds_df['announced_on'] = pd.to_datetime(funding_rounds_df['announced_on'])

funding_rounds_df.columns

Index([u'index', u'company_name', u'country_code', u'state_code', u'region',
       u'city', u'investment_type', u'announced_on', u'raised_amount_usd',
       u'raised_amount', u'raised_amount_currency_code',
       u'post_money_valuation_usd', u'post_money_valuation',
       u'post_money_currency_code', u'investor_count', u'cb_url',
       u'company_uuid', u'funding_round_uuid', u'created_at', u'updated_at',
       u'investor_names', u'investor_uuids', u'MSA_NAME', u'MSA_CODE'],
      dtype='object')

In [16]:
curr_year = datetime.datetime.now().year
year_range = list(map(str, range(curr_year-3, curr_year+1)))

In [17]:
funding_rounds_us_df = funding_rounds_df[ funding_rounds_df['country_code'] == 'USA' ]
funding_rounds_us_df = funding_rounds_us_df[ funding_rounds_us_df['announced_on'].dt.year.isin(year_range) ]

print(funding_rounds_us_df.shape)

funding_rounds_us_df.head()

(21209, 24)


,index,company_name,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,...,investor_count,cb_url,company_uuid,funding_round_uuid,created_at,updated_at,investor_names,investor_uuids,MSA_NAME,MSA_CODE
2,6,"DarkOwl, LLC",USA,CO,Denver,Denver,angel,2016-11-29,1866996.0,NaN,...,NaN,https://www.crunchbase.com/funding-round/owl-c...,ebf194e8-6062-4bef-d33f-bf34cfd95189,e55ebd5b-c7a8-158f-159e-cea5fa36c121,2017-05-24 23:51:23,2017-05-24 23:51:23,{},{},"Denver-Aurora-Lakewood, CO Metro Area",19740.0
3,9,Prieto Battery,USA,CO,Fort Collins,Fort Collins,series_unknown,2016-04-15,1159904.0,NaN,...,NaN,https://www.crunchbase.com/funding-round/priet...,06718a3d-afe6-f023-6f54-f26e1d9b5f24,e55f91a3-6824-f27d-deab-8830b2a7081f,2016-04-18 11:12:50,2016-04-18 11:13:23,{},{},"Fort Collins, CO Metro Area",22660.0
12,38,Fluidic Energy,USA,AZ,Phoenix,Scottsdale,series_unknown,2016-11-28,20000000.0,NaN,...,1.0,https://www.crunchbase.com/funding-round/fluid...,c7d40f02-ec5c-e995-0932-0425404d1ae4,e5694047-8269-c32d-2103-c60d99931f08,2016-11-29 03:40:53,2016-12-01 04:20:06,"{""Asia Climate Partners (ACP)""}",{440fc236-f0ac-2b37-4348-2b793493498a},"Phoenix-Mesa-Scottsdale, AZ Metro Area",38060.0
13,50,Kingwood Energy,USA,MS,Jackson,Jackson,equity_crowdfunding,2016-05-04,NaN,NaN,...,NaN,https://www.crunchbase.com/funding-round/kingw...,9d518767-57b6-b5d7-8662-ade5736728d3,e56bdcfd-5833-d0ba-d809-edfacfec6db9,2016-06-11 06:38:02,2016-06-11 06:38:31,{},{},"Jackson, MS Metro Area",27140.0
15,62,Abeona Therapeutics,USA,OH,Cleveland,Cleveland,grant,2017-10-16,13850000.0,NaN,...,NaN,https://www.crunchbase.com/funding-round/abeon...,aa4e6480-63f8-7609-13f2-dfee6c44ba71,e572753e-6617-47ce-a01b-7d26a55dc198,2017-10-17 04:03:23,2017-10-29 13:44:35,{},{},NaN,NaN


In [20]:
path_to_test_dump = os.path.join(os.environ['DATA_PATH'], 'interim', 'funding_rounds', 'test_data.csv')

msa_codes = [ '45940', '14500', '16820' ] #, '41860', '39340' ]

sample_fr_df = funding_rounds_us_df[ funding_rounds_us_df['MSA_CODE'].isin(msa_codes) ]

sample_fr_df.to_csv(path_to_test_dump, encoding='utf-8', index=False)

sample_fr_df.shape

(231, 24)

In [21]:
from config.api_specs import fields
from src.data.make_dataset import build_investment_flow_df

path_to_dest = os.path.join(os.path.dirname(path_to_test_dump), 'test_result_data.csv')

build_investment_flow_df(path_to_test_dump, path_to_dest, fields['funding_rounds'])


Preparing to filter dataframe. Please wait ..
Filtering dataframe completed!


Preparing to build investment dataframe. Please wait..
Building investment dataframe completed!. Dumping data to /home/banerjee_abhimanyu3/crunchbase/data/interim/funding_rounds/test_result_data.csv



In [22]:
sample_investments = pd.read_csv(path_to_dest, encoding='latin1')

print(sample_investments.shape)
sample_investments.head()

(339, 11)


,index,funding_round_uuid,investment_type,is_lead_investor,money_invested_usd,investee_founded_on,investee_permalink,investee_short_description,investor_founded_on,investor_permalink,investor_short_description
0,0,714fd155-882b-443b-9088-9549fcd48e59,corporate_round,True,NaN,1993-01-01,avery-brewing-co,Avery Brewing Co based in the city of Boulder ...,1890-01-01,mahou-san-miguel,They make 34% of the beer that is produced in ...
1,0,a23d1652-5dee-a6bc-34e6-233056bbef7e,seed,False,NaN,2010-04-22,kindara,Kindara is a platform thatÂ helps women manage...,1995-01-01,sosventures,"SOSV is ""the accelerator"" VC, investing about ..."
2,1,a23d1652-5dee-a6bc-34e6-233056bbef7e,seed,True,NaN,2010-04-22,kindara,Kindara is a platform thatÂ helps women manage...,2010-01-01,boston-seed-capital,"Boston Seed Capital, LLC provides seed stage f..."
3,0,fd2fa0c8-1ef5-bab7-5d51-7f136c599a61,grant,False,25000.0,2014-01-01,eximis-surgical,Eximis Surgical is developing an enabling tech...,2016-07-01,medtechinnovator,MedTech Innovator is a nonprofit global compet...
4,0,fdf21fa7-7b53-7494-d43f-fb2bb88d7254,seed,False,NaN,2010-04-22,kindara,Kindara is a platform thatÂ helps women manage...,2011-09-01,hax-accelerator,HAX is a full stack accelerator for hardware s...
